In [1]:
from confluent_kafka import Consumer
import base64
import json
import io
import numpy as np
import cv2
from PIL import Image
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
# Kafka Configuration
KAFKA_BROKER = "kafka:9092"
TOPIC = "audio_spectrograms"
GROUP_ID = "spectrograms_prediction"

In [3]:
consumer = Consumer({
    "bootstrap.servers": KAFKA_BROKER,
    "group.id": GROUP_ID,
    "auto.offset.reset": "earliest"
})

In [4]:
consumer.subscribe([TOPIC])

In [5]:
# Load model from mounted volume
model = YOLO('/opt/audio_files/full_models/runs/classify/train/weights/last.pt')

In [6]:
# Decode spectrogram
def decode_spectrogram(image_bytes):
    image = Image.open(io.BytesIO(image_bytes)).convert("RGB")  
    return image  

In [7]:
# Consume messages
print("Waiting for spectrogram messages...")
while True:
    msg = consumer.poll(1.0)  # Wait 1 second for a message
    if msg is None:
        continue
    if msg.error():
        print(f"Consumer error: {msg.error()}")
        continue


    headers = msg.headers()
    if headers:
        headers_dict = dict(headers)
        metadata = json.loads(headers_dict.get("metadata", "{}").decode("utf-8"))
        file_name = metadata.get("file_name", "unknown")
        audio_class = metadata.get("class", "unknown")
    else:
        file_name = "unknown"
        audio_class = "unknown"

    print(f"Processing Spectrogram: {file_name}, Class: {audio_class}")

    # Decode Spectrogram (Raw PNG bytes)
    spectrogram_image = decode_spectrogram(msg.value())

    # Run Prediction on YOLO Model
    pred = model(spectrogram_image, verbose=False)
    pred_class = np.argmax(pred[0].probs.data.numpy())

    # Convert predicted class to label
    pred_label = "abnormal" if pred_class == 0 else "normal"
    print(f"Prediction for {file_name}: {pred_label}")

Waiting for spectrogram messages...
Processing Spectrogram: 00000677.wav, Class: normal
Prediction for 00000677.wav: normal
Processing Spectrogram: 00000322.wav, Class: abnormal
Prediction for 00000322.wav: abnormal
Processing Spectrogram: 00000631.wav, Class: normal
Prediction for 00000631.wav: normal
Processing Spectrogram: 00000342.wav, Class: normal
Prediction for 00000342.wav: normal
Processing Spectrogram: 00000855.wav, Class: normal
Prediction for 00000855.wav: normal
Processing Spectrogram: 00000515.wav, Class: normal
Prediction for 00000515.wav: normal
Processing Spectrogram: 00000795.wav, Class: normal
Prediction for 00000795.wav: normal
Processing Spectrogram: 00000166.wav, Class: normal
Prediction for 00000166.wav: normal
Processing Spectrogram: 00000857.wav, Class: normal
Prediction for 00000857.wav: normal
Processing Spectrogram: 00000235.wav, Class: normal
Prediction for 00000235.wav: normal
Processing Spectrogram: 00000876.wav, Class: normal
Prediction for 00000876.wav:

KeyboardInterrupt: 